<a href="https://colab.research.google.com/github/totvslabs/carol-notebooks/blob/main/notebooks/DeleteEmptyUserGroups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Delete Empty User Groups
`This notebook will delete empty user groups within a tenant.`
`The script will ask for the following json on execution:`

```python
{
    "authentication_config" : {
        "username": "username@totvs.com.br",
        "password": "password",
        "organization": "YourOrganization",
        "tenantName": "YourTenantName"
    },
    "script_config" : {
        "carolApp": "YourCarolAppName"
    }
}
```
`You need TENANT ADMIN permissions to run this script.`

#### REQUIREMENTS
`These are the packages the script needs before execution.`

In [ ]:
%%capture
!pip install --quiet pycarol[dataframe]
!pip install --quiet pyarrow
!pip install --quiet python-dotenv
!pip install --quiet curlify
!pip install --quiet git+https://github.com/jnefoussi/pytechfin.git

import json, os
from pycarol import Carol, ApiKeyAuth, PwdAuth, Staging, DataModel
from pycarol import Storage, Connectors
import pandas as pd
import numpy as np
from pycarol.utils.miscellaneous import drop_duplicated_parquet

#### CAROL LOGIN FUNCTIONS
`These are the functions made to login into Carol. They will be the same for all notebooks (ideally) and will use pyCarol.` 

[pyCarol reference](https://github.com/totvslabs/pyCarol)

In [ ]:
def carol_connect(username, password, organization, tenantName):
    print(f"Connecting to Carol tenant {tenantName}... ", end="\n")

    return Carol(domain=tenantName,
                auth=PwdAuth(username, password), organization=organization)

#### SCRIPT FUNCTIONS
`If the script requires more functions to execute, they will be here.`

In [ ]:
def list_groups(login, params):
  return login.call_api(
          "v3/userGroup",
          method="GET",
          content_type="application/x-www-form-urlencoded",
          params=params)

def list_users(login, mdmGroupId):
  query_params_users = {"pageSize": 0}
  body = {"mustList":[{"mdmFilterType":"TYPE_FILTER","mdmValue":"mdmTenantUser"},{"mdmFilterType":"TERM_FILTER","mdmKey":"mdmUserGroupsIds","mdmValue":mdmGroupId}]}
  return login.call_api(
      "v1/users/filterUsers",
      method="POST",
      content_type="application/json",
      params=query_params,
      data=body
  )

def delete_group(mdmGroupId):
    return login.call_api(
          "v3/userGroup/"+mdmGroupId,
          method="DELETE",
          content_type="application/json")

#### CONFIGURATION FILE
`Now you will need to upload the configuration file with the format given above.`

In [ ]:
try:
    from google.colab import files
    config_file = files.upload()
    config_json = json.loads(config_file[next(iter(config_file))].decode("utf-8"))
    config_json_print = json.loads(config_file[next(iter(config_file))].decode("utf-8"))
except:
    with open('./carol.json') as config_file:
        config_json = json.loads(config_file.read())
        config_json_print = json.loads(config_file.read())
    config_file.close()
finally:
    del config_json_print['authentication_config']['password']
    print(json.dumps(config_json_print, indent=2))

#### SCRIPT EXECUTION
`The main execution of the script will happen here.`

In [ ]:
Carol = carol_connect(
    config_json['authentication_config']['username'], 
    config_json['authentication_config']['password'], 
    config_json['authentication_config']['organization'], 
    config_json['authentication_config']['tenantName'])

keep = []
remove = []
next_query = True
counter = -1
offset = 0
pageSize = 500
query_params = {"offset": offset, "pageSize": pageSize}


# Guardando os grupos sem usuários que serão removidos
while(next_query):
  # Consulta de grupos
  resp = list_groups(Carol,query_params)
  if (resp['hits'] and len(resp['hits']) > 0 ):
    for group in resp['hits'] :
      # Consulta usuários
      resp_users = list_users(Carol,group['mdmId'])
      if resp_users['totalHits'] == 0:
        # Grupo sem usuário
        remove.append(group)
      else:
        keep.append(group)
  # Atualização do contador
  if(counter == -1):
    counter = resp['count']
  else:
    counter = counter + resp['count']
  # Atualizando a condição de parada e indice de busca na paginação
  total_hits = resp['totalHits']
  if(total_hits and total_hits > 0):
    if(total_hits == counter):
      next_query = False  
    else:
      offset = counter
      query_params = {"offset": offset, "pageSize": pageSize}
  else:
    next_query = False

#Remoção de grupos
for group in remove:
  try:
    result = delete_group(group['mdmId'])
    #  print(result)
    if result['success']:
      print('Grupo '+ group['mdmName'] + ' removido com sucesso.')
  except:
    print('ERRO ao remover grupo '+ group['mdmName'] )